In [26]:
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim
import requests

import matplotlib.cm as cm
import matplotlib.colors as colors
from bs4 import BeautifulSoup
import xml
import folium

print("All libs imported")

All libs imported


### Scrape Wikipedia page using BeautifulSoup and put data of interest in table_columns

In [15]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response= requests.get(url).text
soup= BeautifulSoup(response,'html.parser')
postcode_table= soup.find('table')
table_columns= postcode_table.find_all('td')

### Convert requied data into a DataFrame

In [46]:
postcode=[]
borough=[]
neighborhood=[]

for i in range(0,len(table_columns),3):
    postcode.append(table_columns[i].text.strip())
    borough.append(table_columns[i+1].text.strip())
    neighborhood.append(table_columns[i+2].text.strip())

df= pd.DataFrame(list(zip(postcode,borough,neighborhood)),columns=['Postcode','Borough','Neighborhood'])
df.shape

(287, 3)

### Data Cleaning and Wrangling

In [47]:
# remove not assigned boroughs
df['Borough'].replace('Not assigned',np.nan, inplace=True)
df.dropna(subset=['Borough'], inplace=True)
df.shape

(210, 3)

In [92]:
# Put neighborhoods in one row separated by comma
df= df.groupby(['Postcode','Borough'])['Neighborhood'].apply(' ,'.join).reset_index()
df.shape

(103, 3)

In [117]:
#just one not assigned neighborhood
df['Neighborhood'].replace('Not assigned','Queen\'s park',inplace=True)
df['Borough'].unique()

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       "Queen's Park", 'Mississauga', 'Etobicoke'], dtype=object)

In [100]:
df_geo = pd.read_csv('http://cocl.us/Geospatial_data')
df_geo.rename(columns={'Postal Code':'Postcode'}, inplace=True)
df_geo

Postcode   Latitude  Longitude
0        M1B  43.806686 -79.194353
1        M1C  43.784535 -79.160497
2        M1E  43.763573 -79.188711
3        M1G  43.770992 -79.216917
4        M1H  43.773136 -79.239476
..       ...        ...        ...
98       M9N  43.706876 -79.518188
99       M9P  43.696319 -79.532242
100      M9R  43.688905 -79.554724
101      M9V  43.739416 -79.588437
102      M9W  43.706748 -79.594054

[103 rows x 3 columns]

### Tried this package but it was not working

In [ ]:
import geocoder
# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
    g = geocoder.google('{}, Toronto, Ontario'.format(postcode))
    lat_lng_coords = g.latlng
    print('g={}', g)  

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

In [162]:
dftoronto= pd.merge(df,df_geo, on='Postcode',how='inner')
dftoronto[dftoronto['Borough'].str.contains('Toronto')]

Postcode           Borough  \
37      M4E      East Toronto   
41      M4K      East Toronto   
42      M4L      East Toronto   
43      M4M      East Toronto   
44      M4N   Central Toronto   
45      M4P   Central Toronto   
46      M4R   Central Toronto   
47      M4S   Central Toronto   
48      M4T   Central Toronto   
49      M4V   Central Toronto   
50      M4W  Downtown Toronto   
51      M4X  Downtown Toronto   
52      M4Y  Downtown Toronto   
53      M5A  Downtown Toronto   
54      M5B  Downtown Toronto   
55      M5C  Downtown Toronto   
56      M5E  Downtown Toronto   
57      M5G  Downtown Toronto   
58      M5H  Downtown Toronto   
59      M5J  Downtown Toronto   
60      M5K  Downtown Toronto   
61      M5L  Downtown Toronto   
63      M5N   Central Toronto   
64      M5P   Central Toronto   
65      M5R   Central Toronto   
66      M5S  Downtown Toronto   
67      M5T  Downtown Toronto   
68      M5V  Downtown Toronto   
69      M5W  Downtown Toronto   
70      M5X  Downtown Toronto   
75      M6G  Downtown Toronto   
76      M6H      West Toronto   
77      M6J      West Toronto   
78      M6K      West Toronto   
82      M6P      West Toronto   
83      M6R      West Toronto   
84      M6S      West Toronto   
87      M7Y      East Toronto   

                                         Neighborhood   Latitude  Longitude  
37                                        The Beaches  43.676357 -79.293031  
41                       The Danforth West ,Riverdale  43.679557 -79.352188  
42                     The Beaches West ,India Bazaar  43.668999 -79.315572  
43                                    Studio District  43.659526 -79.340923  
44                                      Lawrence Park  43.728020 -79.388790  
45                                   Davisville North  43.712751 -79.390197  
46                                 North Toronto West  43.715383 -79.405678  
47                                         Davisville  43.704324 -79.388790  
48                        Moore Park ,Summerhill East  43.689574 -79.383160  
49  Deer Park ,Forest Hill SE ,Rathnelly ,South Hi...  43.686412 -79.400049  
50                                           Rosedale  43.679563 -79.377529  
51                        Cabbagetown ,St. James Town  43.667967 -79.367675  
52                               Church and Wellesley  43.665860 -79.383160  
53                                       Harbourfront  43.654260 -79.360636  
54                           Ryerson ,Garden District  43.657162 -79.378937  
55                                     St. James Town  43.651494 -79.375418  
56                                        Berczy Park  43.644771 -79.373306  
57                                 Central Bay Street  43.657952 -79.387383  
58                           Adelaide ,King ,Richmond  43.650571 -79.384568  
59  Harbourfront East ,Toronto Islands ,Union Station  43.640816 -79.381752  
60           Design Exchange ,Toronto Dominion Centre  43.647177 -79.381576  
61                     Commerce Court ,Victoria Hotel  43.648198 -79.379817  
63                                           Roselawn  43.711695 -79.416936  
64                Forest Hill North ,Forest Hill West  43.696948 -79.411307  
65                The Annex ,North Midtown ,Yorkville  43.672710 -79.405678  
66                     Harbord ,University of Toronto  43.662696 -79.400049  
67          Chinatown ,Grange Park ,Kensington Market  43.653206 -79.400049  
68  CN Tower ,Bathurst Quay ,Island airport ,Harbo...  43.628947 -79.394420  
69                    Stn A PO Boxes 25 The Esplanade  43.646435 -79.374846  
70             First Canadian Place ,Underground city  43.648429 -79.382280  
75                                           Christie  43.669542 -79.422564  
76                       Dovercourt Village ,Dufferin  43.669005 -79.442259  
77                           Little Portugal ,Trinity  43.647927 -79.419750  
78       Brockton ,Exhibition Place ,Parkdale Village  43.63684

### PART 3 OF ASSIGNEMENT- EXPLORE AND CLUSTER

In [123]:
address='Toronto, Canada'

geolocater= Nominatim(user_agent="toronto-exp")
location= geolocater.geocode(address)
lat= location.latitude
lon= location.longitude
print('The geograpical coordinate of the City of Toronto are {}, {}.'.format(lat, lon))

The geograpical coordinate of the City of Toronto are 43.653963, -79.387207.


In [125]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[lat, lon], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(dftoronto['Latitude'], dftoronto['Longitude'], dftoronto['Borough'], dftoronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3199cc',
        fill_opacity=0.3,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [114]:
CLIENT_ID = 'shaded' # your Foursquare ID
CLIENT_SECRET = 'shaded' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KRVXN4W21L5T5HRYMZTBBU2NVQ34QPCLSIYCLYPGFIQV55XC
CLIENT_SECRET:WFCCQH5VJGIPAFPVUR13U15TGBSRNAPKDX4LPV5YRS4M0FQZ


## Explore Toronto Neighborhoods

In [172]:
idx=68
dftoronto.loc[idx]

Postcode                                                      M5V
Borough                                          Downtown Toronto
Neighborhood    CN Tower ,Bathurst Quay ,Island airport ,Harbo...
Latitude                                                  43.6289
Longitude                                                -79.3944
Name: 68, dtype: object

In [173]:
limit=1000
radius=500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    dftoronto.loc[idx, 'Latitude'], 
    dftoronto.loc[idx, 'Longitude'], 
    radius, 
    limit)
response= requests.get(url).json()
#response

In [174]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [166]:
venues = response['response']['groups'][0]['items']    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.location.city','venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

city                                               name  \
0   Toronto  Billy Bishop Toronto City Airport (YTZ) (Billy...   
1   Toronto                                      Porter Lounge   
2   Toronto                                    Toronto Harbour   
3   Toronto                                  Billy Bishop Café   
4   Toronto                        Air Canada Check-In Counter   
5   Toronto                                             Gate 8   
6   Toronto                                          Crew Room   
7   Toronto                                         Market@416   
8   Toronto                                      Want Passport   
9   Toronto                   Porter Airlines Check-In Counter   
10  Toronto                                 Porter FBO Limited   
11  Toronto                                          HeliTours   
12  Toronto                           ORNGE - Toronto Air Base   
13      NaN                               Edward Hanlan Statue   
14  Toronto                               Hanlan's Point Ferry   

            categories        lat        lng  
0              Airport  43.631541 -79.395868  
1       Airport Lounge  43.630680 -79.395756  
2      Harbor / Marina  43.633045 -79.396484  
3   Airport Food Court  43.631132 -79.396139  
4     Airport Terminal  43.631226 -79.395987  
5         Airport Gate  43.631536 -79.394570  
6       Airport Lounge  43.631360 -79.396107  
7                  Bar  43.631653 -79.394510  
8             Boutique  43.631483 -79.396077  
9      Airport Service  43.631683 -79.395454  
10    Airport Terminal  43.630717 -79.398698  
11     Airport Service  43.630706 -79.398760  
12     Airport Service  43.631662 -79.398166  
13    Sculpture Garden  43.627721 -79.389274  
14       Boat or Ferry  43.627813 -79.389109

In [177]:
#put them on a map
# create map of Toronto using latitude and longitude values
map_venues = folium.Map(location=[dftoronto.loc[idx,'Latitude'], dftoronto.loc[idx,'Longitude']], zoom_start=15)

# add markers to map
for lat, lng, name, cat in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name'],nearby_venues['categories']):
    label = '{}, {}'.format(name, cat)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3199cc',
        fill_opacity=0.3,
        parse_html=False).add_to(map_venues)  
    
map_venues